In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import bs4 as bs4
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import urllib

In [21]:
driver = webdriver.Edge()
driver.get('https://spsepchtr.mx.toyota.co.jp/kN0k/V2/servlet/jp.co.toyota.pv2y580.servlet.CCV2Y580')
user_input = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[2]/div/div/form/div[2]/div[1]/table/tbody/tr[1]/td[2]/input')
user_input.send_keys('12062B5@97221')
pwd_input = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div/form/div[2]/div[1]/table/tbody/tr[2]/td[2]/input')
pwd_input.send_keys('Gpauto555#')
pwd_input.send_keys(Keys.ENTER)

In [22]:
confirm_login_button = driver.find_element(By.XPATH,'/html/body/div/div[2]/div/div[2]/input[1]')
confirm_login_button.click()

In [23]:
# Wait for the new window/tab and then switch to it
window_handles = driver.window_handles
driver.switch_to.window(window_handles[-1])

In [24]:
#Get into the menu to search parts.
iframe = driver.find_element(By.XPATH,'/html/body/div/iframe')
driver.switch_to.frame(iframe)
partSearch_button = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[3]/div[1]/fieldset/button[2]')
partSearch_button.click()

In [25]:
#Get into iframe that contain input box
driver.switch_to.default_content()
iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
driver.switch_to.frame(iframe)

<font color='yellow' size = 5>Inventory</font>

In [32]:
GPInventory = pd.read_excel(r'/Users/krichnorakhun/Documents/GP/inventory/GP.xlsx')
GPInventory = GPInventory.loc[GPInventory['ItemGroup'].apply(lambda x : str(x).find('TOYOTA'))>=0][['ItemCode']]

In [33]:
GPInventory = GPInventory.drop_duplicates()

In [34]:
GPInventory.loc[:,'ItemCode'] = GPInventory['ItemCode'].apply(lambda x : x[4:] )

In [35]:
GPInventory = GPInventory.loc[GPInventory['ItemCode'].apply(lambda x : len(x)).isin([10,12])]

In [36]:
GPInventory.columns = ['Code']

In [37]:
GPInventory = GPInventory.reset_index()

In [38]:
GPInventory = GPInventory.drop(['index'], axis=1)

<font color='yellow' size =5>Load from missing images</font>

In [7]:
GPInventory = pd.read_csv(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/temp/missingImage.csv')

In [8]:
GPInventory = GPInventory.drop(['file','size'], axis=1)

In [9]:
GPInventory.columns = ['Code']

In [10]:
GPInventory

,Code
0,687500K010
1,1985011040
2,5534647010
3,1111574120
4,8521428090
...,...
186,9091901259
187,1111528040
188,222150D080
189,819100K010


<font color='yellow' size=5> Scrapping Part </font>

In [11]:
def clickInput10(x):
    #Get input box
    part_1 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[1]')
    part_2 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[2]')
    part_3 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[3]')
    part_1.send_keys(x[:6])
    part_2.send_keys(x[6:11])
    #part_3.send_keys(x[11:])
    part_3.send_keys(Keys.ENTER)
    Run_Botton = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[10]')
    Run_Botton.click()
    return

In [12]:
def clickInput12(x):
    #Get input box
    part_1 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[1]')
    part_2 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[2]')
    part_3 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[3]')
    part_1.send_keys(x[:6])
    part_2.send_keys(x[6:11])
    part_3.send_keys(x[11:])
    part_3.send_keys(Keys.ENTER)
    Run_Botton = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[10]')
    Run_Botton.click()
    return

In [13]:
def getResult_1(x,output):
    tempData = pd.DataFrame()
    soup = bs4.BeautifulSoup(x)
    tableInSoup = soup.find_all('table',{'class':'excelLikeList'})
    tempData.loc[0, 'PartID'] = tableInSoup[0].find_all('div', { 'class':'side'})[0].text
    tempData.loc[0, 'Part Description'] = tableInSoup[0].find_all('div', { 'class':'side'})[1].text
    output = pd.concat([output,tempData], ignore_index=True)
    return(output)

In [14]:
def clickNewInput():
    newFind = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[1]')
    newFind.click()


In [15]:
def getResult_2(data,output):
    soup = bs4.BeautifulSoup(data)
    tableInSoup = soup.find_all('table',{'class':'excelLikeList'})
    tempData = pd.DataFrame()
    for x in tableInSoup[1].find_all('tr'):
        tempData.loc[0,'PartID'] = x.find_all('td')[1].text
        tempData.loc[0,'PNC'] = x.find_all('td')[2].text
        tempData.loc[0,'Count'] = x.find_all('td')[3].text
        tempData.loc[0,'Catalogue'] = x.find_all('td')[4].text
        tempData.loc[0,'Model'] = x.find_all('td')[5].text
        tempData.loc[0,'Fr-To'] = x.find_all('td')[6].text
        tempData.loc[0,'Model(Details)'] = x.find_all('td')[7].text
        output = pd.concat([output, tempData], ignore_index=True)
    return(output)
    #print(x.find_all('td')[1].text)

In [16]:
def getResult_2pics(data,output,imgName,thirdTable):
    soup = bs4.BeautifulSoup(data)
    tableInSoup = soup.find_all('table',{'class':'excelLikeList'})
    
    tempData_List = []
    for x in tableInSoup[1].find_all('tr'):
        # tempData.loc[0,'PartID'] = x.find_all('td')[1].text
        # tempData.loc[0,'PNC'] = x.find_all('td')[2].text
        # tempData.loc[0,'Count'] = x.find_all('td')[3].text
        # tempData.loc[0,'Catalogue'] = x.find_all('td')[4].text
        # tempData.loc[0,'Model'] = x.find_all('td')[5].text
        # tempData.loc[0,'Fr-To'] = x.find_all('td')[6].text
        # tempData.loc[0,'Model(Details)'] = x.find_all('td')[7].text
        
        tempData_List.append({
        'PartID': x.find_all('td')[1].text,
        'PNC': x.find_all('td')[2].text,
        'Count': x.find_all('td')[3].text,
        'Catalogue': x.find_all('td')[4].text,
        'Model': x.find_all('td')[5].text,
        'Fr-To' : x.find_all('td')[6].text,
        'Model(Details)' : x.find_all('td')[7].text
        })

        #output = pd.concat([output, tempData], ignore_index=True)
        
        #for add to Third Table
        PartID = x.find_all('td')[1].text
        PNC = x.find_all('td')[2].text
    tempData = pd.DataFrame(tempData_List)
    output = pd.concat([output, tempData], ignore_index=True)        
    
    #Click First Row
    firtRowToClickID = soup.find_all('table',{'id':'F10_tblListInnerSearch'})[0].find_all('tr')[0].get('id')
    driver.find_element(By.ID,firtRowToClickID).click()

    #Click button to open new window to show picture
    driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[8]').click()
    
    # Wait for a new window/tab to open
    wait = WebDriverWait(driver, 60)
    current_windows = driver.window_handles
    wait.until(lambda driver: len(driver.window_handles) > len(current_windows))

    # Switch to the new window/tab
    new_window_handles = driver.window_handles
    driver.switch_to.window(new_window_handles[-1])
        
    #Get into iframe where picture located.
    iframe = driver.find_element(By.XPATH,'/html/body/div/iframe')
    driver.switch_to.frame(iframe)

    # Wait for the image to be present
    wait = WebDriverWait(driver, 30)  # Adjust the timeout as needed
    img_element = wait.until(EC.presence_of_element_located((By.ID, 'figImgFig')))

    # Now retrieve the image URL
    img_url = img_element.get_attribute('src')
    urllib.request.urlretrieve(img_url,'images/'+imgName+'.png')

    #Get PNC list on image and its location
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    
    tempdata2_List = []
    width = float(soup.find_all('img',{'id':'figImgFig'})[0].get('style').split(';')[1].split(':')[1].replace('px','').strip())
    height = float(soup.find_all('img',{'id':'figImgFig'})[0].get('style').split(';')[2].split(':')[1].replace('px','').strip())
    for pnc in soup.find_all('div',{'id':'figInner'})[0].find_all('div'):
        top = float(pnc.get('style').split(';')[0].split(':')[1].replace('px', ''))
        left = float(pnc.get('style').split(';')[1].split(':')[1].replace('px', ''))
        fontsize = float(pnc.get('style').split(';')[2].split(':')[1].replace('px',''))
        lineheight = float(pnc.get('style').split(';')[3].split(':')[1].replace('px',''))
        transform = pnc.get('style').split(';')[4].split(':')[1].strip()        
        text = pnc.text
         # Append to the list as a dictionary
        tempdata2_List.append({
        'PartID': PartID,
        'PNC': PNC,
        'Top': top,
        'Left': left,
        'Fontsize':fontsize,
        'LineHeight':lineheight,
        'Transform':transform,
        'Width':width,
        'Height':height,
        'Text': text
        })
    tempData2 = pd.DataFrame(tempdata2_List)
    thirdTable = pd.concat([thirdTable, tempData2], ignore_index=True)

    #Close window and switch back to previous window.
    driver.close()
    # Wait for the new window/tab and then switch to it
    window_handles = driver.window_handles
    driver.switch_to.window(window_handles[-1])

    iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
    driver.switch_to.frame(iframe)
    
    return output,thirdTable
    #print(x.find_all('td')[1].text)

<font color='yellow' size=5> simulate save picture </font>

In [21]:
clickInput12('687500K010')

In [22]:
WebDriverWait(driver,60)
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [23]:
firtRowToClickID = soup.find_all('table',{'id':'F10_tblListInnerSearch'})[0].find_all('tr')[0].get('id')

In [24]:
driver.find_element(By.ID,firtRowToClickID).click()

In [25]:
#Click to open new window to show picture
driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[8]').click()
# Wait for a new window/tab to open
wait = WebDriverWait(driver, 60)
current_windows = driver.window_handles
wait.until(lambda driver: len(driver.window_handles) > len(current_windows))

True

In [34]:
# Switch to the new window/tab
new_window_handles = driver.window_handles
driver.switch_to.window(new_window_handles[-1])

In [35]:
#Get into iframe where picture located.
iframe = driver.find_element(By.XPATH,'/html/body/div/iframe')
driver.switch_to.frame(iframe)

In [36]:
# Wait for the image to be present
wait = WebDriverWait(driver, 30)  # Adjust the timeout as needed
img_element = wait.until(EC.presence_of_element_located((By.ID, 'figImgFig')))

# Now retrieve the image URL
img_url = img_element.get_attribute('src')
print(img_url)

https://spsepchtr.mx.toyota.co.jp/kN0k/CONTENTS/HBUNKAI/113494B/113494B.PNG


In [37]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [54]:
width = float(soup.find_all('img',{'id':'figImgFig'})[0].get('style').split(';')[1].split(':')[1].replace('px','').strip())

In [60]:
height = float(soup.find_all('img',{'id':'figImgFig'})[0].get('style').split(';')[2].split(':')[1].replace('px','').strip())

492.0

In [31]:
pnc = soup.find_all('div',{'id':'figInner'})[0].find_all('div')[0]

In [51]:
pnc.get('style')#.split(';')[4].split(':')[1].strip()

'top: 243.024px; left: 156.81px; font-size: 8.09647px; line-height: 8.09647px; transform: scale(1.1, 1.25);'

In [412]:
tempData2 = pd.DataFrame()
for pnc in soup.find_all('div',{'id':'figInner'})[0].find_all('div'):
    tempData2['PartID'] = PartID
    tempData2['PNC'] = PNC
    tempData2['Top'] = float(pnc.get('style').split(';')[0].split(':')[1].replace('px',''))
    tempData2['Left'] = float(pnc.get('style').split(';')[1].split(':')[1].replace('px',''))
    
    tempData2['Text'] = pnc.text
    
    

top: 243.024px; left: 156.81px; font-size: 8.09647px; line-height: 8.09647px; transform: scale(1.1, 1.25);
156.81
04112
top: 6.02412px; left: 155.81px; font-size: 8.09647px; line-height: 8.09647px; transform: scale(1.1, 1.25);
155.81
04111


In [395]:
soup.find_all('div',{'id':'figInner'})[0].find_all('div')[0].get('style')

'top: 243.024px; left: 156.81px; font-size: 8.09647px; line-height: 8.09647px; transform: scale(1.1, 1.25);'

In [92]:
driver.close()

In [27]:
# Wait for the new window/tab and then switch to it
window_handles = driver.window_handles
driver.switch_to.window(window_handles[-1])

In [28]:
#Get into iframe that contain input box
iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
driver.switch_to.frame(iframe)

In [210]:
bs4.BeautifulSoup(driver.page_source).find_all('title')

[<title>Part Number Application to Models</title>]

In [159]:
soup.find_all('title')

[<title>Part Number Application to Models</title>]

<font color='yellow' size=5>RUN</font>

In [17]:
MainTable = pd.DataFrame()
SecondTable = pd.DataFrame()
ThirdTable = pd.DataFrame()

In [18]:
def clearCheckbox():
    driver.find_element(By.ID,'F10_chkSmksKbnJA').click()
    driver.find_element(By.ID,'F10_chkSmksKbnEU').click()
    driver.find_element(By.ID,'F10_chkSmksKbnNA').click()

In [26]:
for index, row in GPInventory.iloc[165:].iterrows():
    
    if len(row['Code'].strip())==10:
        clickInput10(row['Code'].strip())
    else:
        clickInput12(row['Code'].strip())
    
    wait = WebDriverWait(driver, 60) 
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)

    try:
        wait.until(EC.visibility_of_element_located((By.ID, 'eltColF10_divListSelectHinban4')))
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)
        if soup.find_all('div',{'id':'eltColF10_divListSelectHinban4'})[0].text != 'ไม่มีใช้งาน':
            wait.until(EC.visibility_of_element_located((By.ID, 'eltRowF10_divListSearch1')))
       
        #time.sleep(1)
        data = driver.page_source
        MainTable = getResult_1(data,MainTable)
        if soup.find_all('div',{'id':'eltColF10_divListSelectHinban4'})[0].text != 'ไม่มีใช้งาน':
            SecondTable, ThirdTable = getResult_2pics(data,SecondTable,row['Code'].strip(),ThirdTable)
        print(index)
        
    except TimeoutException:
        print("Timeout occurred! Performing alternative steps.")

    finally:
        clickNewInput()
        clearCheckbox()
        

165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [473]:
#Get into iframe that contain input box
driver.switch_to.default_content()
iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
driver.switch_to.frame(iframe)

In [43]:
clickNewInput()

In [74]:
MainTable1 = MainTable.drop_duplicates()

In [75]:
MainTable1.to_excel(r'/Users/krichnorakhun/Documents/GP/inventory/EPC1_w_images.xlsx', index=False)

In [76]:
SecondTable1 = SecondTable.drop_duplicates()

In [77]:
SecondTable1.to_excel(r'/Users/krichnorakhun/Documents/GP/inventory/EPC2_w_images.xlsx', index=False)

In [78]:
ThirdTable1 = ThirdTable.drop_duplicates()

In [79]:
ThirdTable1.to_csv(r'/Users/krichnorakhun/Documents/GP/inventory/EPC3_w_images.csv', index=False)

In [73]:
MainTable

,PartID,Part Description
0,04111-0E042,ไม่มีใช้งาน
1,04111-30544,"ชุดปะเก็น, การซ่อมใหญ่เครื่องยนต์"
2,04152-YZZA1,ไม่มีใช้งาน
3,04152-YZZA5,ไม่มีใช้งาน
4,04111-0C136,ไม่มีใช้งาน
...,...,...
4351,PZ046-0KS65,ไม่มีใช้งาน
4352,PZ057-0K022-00,ไม่มีใช้งาน
4353,PZT01-8702L,ไม่มีใช้งาน
4354,PZT01-8742L,ไม่มีใช้งาน


In [58]:
SecondTable

,PartID,PNC,Count,Catalogue,Model,Fr-To,Model(Details)
0,04111-30544,04111,01,711350,HIACE,201005- 201309,2KDFTV..KDH222..TH
1,04112-0L191,04112,01,6713A0,FORTUNER,201410- 201611,"2KDFTV..KUN50,60..IDN,MA,PHL,SA,VN*130"
2,04112-0L191,04112,01,672380,HILUX,201410- 201601,"2KDFTV..KUN25..IDN,MA2KDFTV..KUN35..DLX..PHL2K..."
3,04112-0L191,04112,01,672390,HILUX,201505- 201707,"2KDFTV..KUN125,135..HDCR"
4,04226-0L010,04226,01,611260,INNOVA/KIJANG INNOVA,200507- 200708,2KDFTV..KUN40..IND*163 OR *184
...,...,...,...,...,...,...,...
49345,53124-YP200,53122F,01,149330,YARIS/VIOS/LIMO (4D),202009- 202207,NSP152MLM PACKAGE&ACTIVE SAFETY PACKAGE-WITH
49346,53124-YP200,53124C,01,14C310,VIOS (4D),202007-,NSP151MLM PACKAGE
49347,53124-YP200,53124C,01,14C310,VIOS (4D),202103-,NSP151..CVTGR SPORT&MLM PACKAGE
49348,53124-YP200,53124C,01,14E310,LIMO/VIOS (4D),202103-,NSP151..GMLM PACKAGE


In [75]:
SecondTable.loc[SecondTable['PartID']=='27060-0C021']

,PartID,PNC,Count,Catalogue,Model,Fr-To,Model(Details)
21770,27060-0C021,27020,01,611260,INNOVA/KIJANG INNOVA,201003- 201204,"1TRFE,2TRFE..TGN4#..GCC,IDN,MA,PHL,TH,VN12V 80A"
21771,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201107- 201204,2TRFE..TGN4112V 80A
21772,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201107- 201212,"1TRFE..TGN40..(GL,STD)..VN1TRFE..TGN40..4FC..D..."
21773,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201107- 201304,"1TRFE..TGN40..IDN,TH1TRFE..TGN40..5F..DLX..VN1..."
21774,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201204- 201304,2TRFE..TGN4112V 80A
21775,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201107- 201603,"1TRFE..TGN40..MA,PHL12V 80A"
21776,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201304- 201603,2TRFE..TGN4112V 80A
21777,27060-0C021,27020,01,671380,FORTUNER,201003- 201204,"2TRFE..TGN51,6112V 80A"
21778,27060-0C021,27020,01,6713A0,FORTUNER,201107- 201204,"2TRFE..TGN51,61..GCC,PHL12V 80A"
21779,27060-0C021,27020,01,6713A0,FORTUNER,201107- 201304,"2TRFE..TGN51,61..IDN,TH,VN12V 80A"


In [80]:
ThirdTable

,PartID,PNC,Top,Left,Fontsize,LineHeight,Transform,Width,Height,Text
0,04111-30544,04111,243.02400,156.8100,8.09647,8.09647,"scale(1.1, 1.25)",339.0,492.0,04112
1,04111-30544,04111,6.02412,155.8100,8.09647,8.09647,"scale(1.1, 1.25)",339.0,492.0,04111
2,04112-0L191,04112,13.02460,157.8100,8.09856,8.09856,"scale(1.1, 1.25)",339.0,493.0,04111
3,04112-0L191,04112,28.02460,157.8100,8.09856,8.09856,"scale(1.1, 1.25)",339.0,493.0,04112
4,04226-0L010,04226,264.73400,476.6330,7.34137,7.34137,"scale(0.95, 1.1)",650.0,448.0,90466-T0041
...,...,...,...,...,...,...,...,...,...,...
128012,99367-J1960,16361A,291.77100,34.7084,7.08441,7.08441,"scale(1.1, 1.25)",339.0,492.0,FIG 16-01
128013,99367-J1960,16361A,130.77100,185.7080,7.08441,7.08441,"scale(1.1, 1.25)",339.0,492.0,REFER TO
128014,99367-J1960,16361A,136.77100,185.7080,7.08441,7.08441,"scale(1.1, 1.25)",339.0,492.0,FIG 11-06
128015,99367-J1960,16361A,130.77100,264.7080,7.08441,7.08441,"scale(1.1, 1.25)",339.0,492.0,REFER TO
